In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import pickle as pkl
import glob
import re
from proxy_requests import ProxyRequests
import random

In [2]:
# run get_incident_file_list function
# to output all recorded possible indident ids into one list
from my_functions import get_incident_file_list
# run get_incident_std_list function
# to output possible indident ids from scraping methods into one list
from my_functions import get_incident_std_list
# run get_logged_incident_ids_list function
# to output all logged indident ids into one list
from my_functions import get_logged_incident_ids_list
# run get_original_incident_ids_list function
# to output all indident ids from original file into one list
from my_functions import get_original_incident_ids_list
# run get_id_status_queue function
# to output all potential ids
# that have not yet been logged
# up to an inputted maximum id
# minimum id has been set as site minimum
from my_functions import get_id_status_queue
# function outputs a dataframe of all manually requested status codes
# that have been confirmed as 200 or 404
from my_functions import get_status_code_df
# function takes a list input to catalogue in the incident directory without duplicates
# when requests are made later, they will only be for non-catalogued IDs
# removing any potential id that is already in the incident folder
from my_functions import get_id_file_add_queue
# function to scrape whatismybrowser.com and return a popular header at random
from my_functions import get_popular_header
# function takes in a url as an input and assigns a random proxy and a random header
# function returns status_code as 0 indice
# function returns get_raw as 1 indice
from my_functions import request_with_random_header

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 1000)

In [4]:
incident_std_list = get_incident_std_list()
logged_ids_list = get_logged_incident_ids_list()
# inputting ids from logged_ids_list and incident_std_list
# that are not already in the incident directory into incident directory into the incident directory
# making sure every catalogued (scraped and downloaded) ID is in the incident folder,
# all scraped ids and all logged ids will be accounted for
get_id_file_add_queue(logged_ids_list + incident_std_list)

id_file_add_queue: 0



In [5]:
# # creating a while loop to obtain the maximum_id to input into status_id function
# status_code = None
# while status_code!=200:
#     request = request_with_random_header('https://www.gunviolencearchive.org/last-72-hours')
#     status_code = request.get_status_code()
#     print(str(status_code) + ', ' + str(request.get_proxy_used()))
# raw_html = request.get_raw()
# maximum_id = int(str(str(raw_html).split('<a href="/incident/')[1]).split('>')[0][:-1])
# print('maximum_id: ' + str(maximum_id) + '\n')
# # looping through all unaccounted for potential ids to get remaining status codes
# # retrieving status codes for all potential ids that have not yet been logged
# # 200 codes will be sent to incident directory, and then to scraper later
# id_status_queue = get_id_status_queue(maximum_id)
# for possible_id in id_status_queue:
#     request = request_with_random_header('https://www.gunviolencearchive.org/incident/' + str(possible_id))
#     id_status_code = request.get_status_code()
#     with open('../Data Sources/brute_force_id_status_codes.txt', 'a+') as f:
#         f.write("\n" + str(possible_id) + ', ' + str(id_status_code))
# #         print(str(possible_id) + ', ' + str(id_status_code) + ', ' + str(request.get_proxy_used()))

In [6]:
status_code_df = get_status_code_df()
# make a new add_list for urls that have received 200 codes above
status_200_ids = list(status_code_df[status_code_df['status_code']==200]['id'].unique())
# inputting status_200_ids into incident directory
# no status 200 id should already be in the incident directory
# making sure every manually requested id that was successful
# is in the incident folder
get_id_file_add_queue(status_200_ids)

id_file_add_queue: 2963



In [7]:
incident_file_list = get_incident_file_list()
original_ids_list = get_original_incident_ids_list()
# adding all ids from incident file list that are not in the original ids list into the scraper queue
# ids that have already been scraped are ok because of cache
id_scrape_queue = deepcopy(list(set(incident_file_list) - set(original_ids_list)))

with open('../Pickles/incident_id_scrape_queue.pkl', 'wb') as f:
    pkl.dump(id_scrape_queue, f)

print('id_scrape_queue: ' + str(len(id_scrape_queue)))

id_scrape_queue: 18000


In [ ]:
incident_file_list = get_incident_file_list()
# catalogue all possible incident_ids into a csv file
df = pd.DataFrame(incident_file_list, columns = ['possible_incident_ids.csv'])
df.to_csv('../Data Sources/possible_incident_ids.csv')